In [1]:
import pinecone
from tqdm.autonotebook import tqdm
import os
import sys
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Pinecone

/Users/skarlekar/anaconda3/envs/openai/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY=os.environ['PINECONE_API_KEY']
PINECONE_ENV=os.environ['PINECONE_ENV']
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

In [3]:
index_name = 'uw-guide-index'
if index_name not in pinecone.list_indexes():
    print("Underwriting Guide index unavailable. Exiting!")
    sys.exit()
else:
    print("Found Underwriting Guide!")

Found Underwriting Guide!


In [5]:
embeddings = OpenAIEmbeddings()

In [6]:
vector_store = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

In [7]:
user_question = (
    "What is ACheck, when is it used, what are the requirements for using ACheck?"
)
resultset = vector_store.similarity_search(user_question)
print(len(resultset))
for x in resultset:
    print(x)

4
page_content='When do you use the ACheck application?\nAs soon as you receive an application request and the necessary tax identification numbers and/or social security numbers, enter the information into ACheck.\nIf you have not yet identified all parties at this stage, you must enter all Key Principals, Principals, and Guarantors of the Borrower into ACheck as soon as you identify them.\nIf more than 90 days pass between the initial ACheck and Commitment Date, you must repeat an ACheck.\nHow do you view ACheck results?\nThe ACheck application will provide either a “Continue Processing” or “Do Not Process” electronic response instantaneously.\n“Continue Processing” Response\nYou must receive a “Continue Processing” response in order to proceed with the application.\nThis does not mean that the Borrower, Key Principal, Principal, Guarantor, or Principal is approved; you are still required to complete full Mortgage Loan credit underwriting.\n“Do Not Process” Response' metadata={'sourc

In [9]:
llm = ChatOpenAI(temperature=0, model='gpt-4')
chain = load_qa_with_sources_chain(llm, chain_type="stuff")
response = chain.run(input_documents=resultset, question=user_question)
print(response)

ACheck is a lender due diligence performed for the Borrower, Key Principal, and Principal using the ACheck application. It is used as soon as an application request is received along with the necessary tax identification numbers and/or social security numbers. All Key Principals, Principals, and Guarantors of the Borrower must be entered into ACheck as soon as they are identified. If more than 90 days pass between the initial ACheck and Commitment Date, an ACheck must be repeated. The ACheck application will provide either a “Continue Processing” or “Do Not Process” electronic response instantaneously. A "Continue Processing" response must be received in order to proceed with the application. This does not mean that the Borrower, Key Principal, Principal, Guarantor, or Principal is approved; full Mortgage Loan credit underwriting is still required.
SOURCES: docs/Part I Chapter 3 - Public - 2023-05-15.pdf, docs/Part III Chapter 7 - Public - 2023-05-15.pdf
